_Última modificación: 09 de noviembre de 2024_
# Práctica 3: Identificación de frases clave y resumen automático de texto
**Hernández Jiménez Erick Yael**: 2023630748.

Escuela Superior de Cómputo.

Ingeniería en inteligencia Artificial. 5BV1.

Tecnologías para el Procesamiento de Lenguaje Natural.



## Resumen
En este cuaderno se usará el procesamiento de cuerpos para realizar un resumen automático extractivo de 3 documentos tras normalizarlos. A continuación se enlistan las bibliotecas utilizadas.

In [1]:
import nltk     # Biblioteca con las herramientas utilizadas para la manipulación y procesamiento de los documentos
from nltk.tokenize import word_tokenize, sent_tokenize  # Submódulo para tokenizar
from nltk.tokenize import RegexpTokenizer  # Para generar el filtro
nltk.download('punkt_tab')
nltk.download('stopwords')  # Lista con stopwords en distintos idiomas
import random               # Para números aleatorios
import math                 # Para operaciones matemáticas complejas

[nltk_data] Downloading package punkt_tab to /home/hjey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hjey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Generación de cuerpo de documentos
Extrayendo las primeras 3 cartas del libro de Frankstein, desde el enlace [“https://www.gutenberg.org/ebooks/84”](“https://www.gutenberg.org/ebooks/84”), los textos se guardaron manualmente como archivos de texto en la carpeta docs como [Carta_1](./docs/Carta_1.txt), [Carta_2](./docs/Carta_2.txt) y [Carta_3](./docs/Carta_3.txt). A partir de estos, incluimos un enunciado de cada uno.

In [2]:
with open("docs/Carta_1.txt", 'r', encoding="utf-8") as file:
    carta_1: str = file.read()
with open("docs/Carta_2.txt", 'r', encoding="utf-8") as file:
    carta_2: str = file.read()
with open("docs/Carta_3.txt", 'r', encoding="utf-8") as file:
    carta_3: str = file.read()

Tokenizamos en enunciados para, luego, escoger un enunciado al azar de cada capítulo y así generar el documento

In [3]:
# Tokenizamos por enunciados
original_1 = sent_tokenize(carta_1)
original_2 = sent_tokenize(carta_2)
original_3 = sent_tokenize(carta_3)

'''
corpus                                      Nuestra variable con el corpus
        += str(                             Forzamos el tipo del iterable a una cadena de caracteres
            original_x                      Para el originial número 'x'...
                [random.randint(            escogemos un número aleatorio...
                    0,                      entre los índices 0...
                    len(original_x)-1)])      y la longitud de los enunciados en el original número 'x'
        + " "                               Y concatenamos un espacio para distinguir entre enunciados extraídos
'''
corpus: str = str(original_1[random.randint(0, len(original_1)-1)]) + " "
corpus += str(original_2[random.randint(0, len(original_2)-1)]) + " "
corpus += str(original_3[random.randint(0, len(original_3)-1)])
print(corpus)

Six years have passed since I resolved on my present undertaking. There is something at work in my soul which I do not understand. What can stop the determined heart and resolved will of man?


Con esto, tenemos el corpus sobre el que trabajaremos a lo largo de la práctica

## Normalización de textos general
El flujo que se usará, y su justificación se explicará a continuación:
1. Conversión a minúsculas: para evitar redundancias en el contenido significativo del cuerpo
2. Elimininación de espacios, números, signos de puntuación y el caracter '—': Estos caracteres se encuentra en los 3 archivos originales, siendo de carácter visual para separar diálogos y contextos en las frases. No aporta contenido al proceso de ninguno de los algoritmos por lo que su eliminación reducirá el análisis. Cabe mencionar que el caracter '—' es disntinto de '-', siendo el último relevante para la generación del resumen debido a que cambia el significado de las palabras adyacentes, por lo que se mantiene en el cuerpo.

In [4]:
# Conversión a minúsculas
corpus = corpus.lower()

# Tokenizamos por enunciados y forzamos el tipo de la varible a una lista
documentos = sent_tokenize(corpus)

# Eliminamos los caracteres que no nos interesen de cada documento sin perder la estructura u orden
for i, doc in enumerate(documentos):
# Para cada documento 'doc' con índice 'i' en los elementos enumerados de 'documentos'
    documentos[i] = ''.join([char for char in doc if char.isalpha() or char == ' ' or char == '-'])
    # Al elemento [i] lo reasignamos como la unión con el caractér vacío '' por cada caracter 'char'
    # si 'char' es una letra o es un espacio o el guión '-'

# Tokenizamos por palabras
palabras_1 = word_tokenize(documentos[0])
palabras_2 = word_tokenize(documentos[1])
palabras_3 = word_tokenize(documentos[2])

# Imprimimos los resultados
print(f"Carta 1:\n{documentos[0]}\n{palabras_1}\n\nCarta 2:\n{documentos[1]}\n{palabras_2}\n\nCarta 3:\n{documentos[2]}\n{palabras_3}\n")

Carta 1:
six years have passed since i resolved on my present undertaking
['six', 'years', 'have', 'passed', 'since', 'i', 'resolved', 'on', 'my', 'present', 'undertaking']

Carta 2:
there is something at work in my soul which i do not understand
['there', 'is', 'something', 'at', 'work', 'in', 'my', 'soul', 'which', 'i', 'do', 'not', 'understand']

Carta 3:
what can stop the determined heart and resolved will of man
['what', 'can', 'stop', 'the', 'determined', 'heart', 'and', 'resolved', 'will', 'of', 'man']



## Resumen automático extractivo de texto

### TF-IDF - NLTK
Siguiendo el ejemplo de [TURING](https://www.turing.com/kb/5-powerful-text-summarization-techniques-in-python), aplicaremos el algoritmo TF-IDF:

In [5]:
# Crearemos clases que conserven estos datos para analizarlos posteriormente
class TF_IDF():
    r'''
    Clase que contiene el resumen por TF-IDF de una serie de tokens:
    '''
    def __init__(self, tokens, corpus, lang: str = "english"):
        r'''
        # __init__
        - tokens: Any = iterable con los tokens del documento sobre el que se aplicará el algoritmo TF-IDF
        - lang: str = nombre del lenguaje a partir del cual se eliminarán las stop-words que NLTK tiene por defecto 
        '''
        self.tokens = tokens
        self.corpus = corpus
        self.stopWords: set = nltk.corpus.stopwords.words("english") # Definimos el conjunto de stopwords
        self.tablaFrec: dict = {}   # Inicializamos con un diccionario vacío
        self.pesoEnun: dict = {}    # Inicializamos con un diccionario vacío
        self.totalPesos: int = 0    # Número descriptivo del total de los pesos
        self.promedio: int = 0      # Número descriptivo con el promedio de los pesos
        self.resumen: list = []     # Lista vacía para almacenar las palabras más frecuentes que conforman al resumen
    
    def calcularTF(self) -> None:
        r'''
        # generarTablaFreq
        Genera la tabla de frecuencias normalizadas del documento
        '''
        # Conteo de frecuencias
        for palabra in self.tokens:        # Por cada palabra en los tokens...
            if palabra in self.stopWords:  # si la palabra se incluyen en las stopwords...
                continue                # las omitimos y continuamos
            else:                       # de lo contrario...
                if palabra in self.tablaFrec:      # si la palabra ya se incluye en la tabla de frecuencias...
                    self.tablaFrec[palabra] += 1   # aumentamos en 1 el contador
                else:                           # sino...
                    self.tablaFrec[palabra] = 1    # Empezamos el conteo en 1
        # Normalización de frecuencias
        for palabra in self.tablaFrec.keys():
            self.tablaFrec[palabra] /= len(self.tokens)

    def calcularIDF(self) -> None:
        r'''
        #calcularIDF
        Calcular el peso que tiene cada palabra en los enunciados con respecto a todos los documentos
        '''
        num_docs = len(self.corpus)
        for palabra in self.tablaFrec:
        # Por cada palabra en la tabla de frecuencias...
            doc_count = sum(1 for documento in self.corpus if palabra in documento)
            # Aumentamos en uno el contador de documentos que contengan a la palabra de la iteración
            self.pesoEnun[palabra] = math.log(num_docs / (1 + doc_count)) # Se agrega el 1 para suavizar el peso de palabras raras
            # Para luego calcular el valor correspondiente al IDF y asignarlo al diccionario con las palabras del corpus

    def imprimir_tabla(self, tabla: dict) -> None:
        r'''
        #imprimir_tabla
        Imprime la tabla indicada
        '''
        for key in tabla:
            print(f"'{key}':{tabla[key]}")
        print("\n")

    def calcular_TF_IDF(self) -> None:
        self.calcularTF()
        self.calcularIDF()
        tf_idf = {}
        for palabra in self.tablaFrec:
            tf = self.tablaFrec[palabra]
            idf = self.pesoEnun.get(palabra, 0)
            tf_idf[palabra] = tf * idf  # Multiplicación de TF e IDF
        self.resumen = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)

In [6]:
# Aplicamos el algoritmo sobre los documentos
td_idf_1 = TF_IDF(palabras_1, documentos)
td_idf_2 = TF_IDF(palabras_2, documentos)
td_idf_3 = TF_IDF(palabras_3, documentos)

td_idf_1.calcular_TF_IDF()
td_idf_2.calcular_TF_IDF()
td_idf_3.calcular_TF_IDF()

print(f"Documento 1\n{td_idf_1.resumen[0:5]}")
print(f"Documento 2\n{td_idf_2.resumen[0:5]}")
print(f"Documento 3\n{td_idf_3.resumen[0:5]}")

Documento 1
[('six', 0.036860464373469494), ('years', 0.036860464373469494), ('passed', 0.036860464373469494), ('since', 0.036860464373469494), ('present', 0.036860464373469494)]
Documento 2
[('something', 0.03118962370062803), ('work', 0.03118962370062803), ('soul', 0.03118962370062803), ('understand', 0.03118962370062803)]
Documento 3
[('stop', 0.036860464373469494), ('determined', 0.036860464373469494), ('heart', 0.036860464373469494), ('man', 0.036860464373469494), ('resolved', 0.0)]


### Frecuencia de palabras normalizada
De acuerdo con [Matthew Mayo](https://www.kdnuggets.com/2019/11/getting-started-automated-text-summarization.html) y [Dante Sblendorio](https://www.activestate.com/blog/how-to-do-text-summarization-with-python/). Se debe seguir el siguiente algoritmo:
1. Normalizar los documentos.
2.  Por cada palabra en el corpus se cuenta su frecuencia
3.  Se obtiene la frecuencia más alta
4.  Se divide cada frecuencia entre la frecuencia más alta
5.  Por cada documento y cada palabra en el corpus, si la palabra se encuentra en el documento, se aumenta el conteo del enunciado
7.  Se ordenan los enunciados por su puntación y se obtiene el resumen